In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Stabilità interna

Il concetto di stabilità cattura il comportamento dell'evoluzione dello stato del sistema quando questo è perturbato da una condizione di equilibrio: la stabilità descrive se l'evoluzione dello stato che si manifesta dopo una perturbazione da un punto di equilibrio diverge o meno.

### Definizione
Dato un sistema dinamico invariante nel tempo descritto dal vettore di stato $x(t)\in \mathbb{R}^n$, un punto di equilibrio $x_e$, uno stato iniziale $x_0$ e un tempo iniziale $t_0$ se
$$
\forall \, \epsilon \in \mathbb{R}, \, \epsilon > 0 \quad \exists \delta \in \mathbb{R}, \, \delta > 0 : \quad ||x_0-x_e|| < \delta \, \Rightarrow  \, ||x(t)-x_e|| < \epsilon \quad \forall t \ge t_0
$$
che potrebbe essere letto come: se esiste una perturbazione iniziale abbastanza piccola $\delta$ dal punto di equilibrio in modo che l'evoluzione dello stato $x(t)$, dal punto perturbato, non si allontani troppo (più di $\epsilon$) dall'equilibrio stesso,
allora il punto di equilibrio è stabile.

Se accade anche che $\lim_{t\to\infty}||x(t)-x_e|| = 0$, che si può leggere come: l'evoluzione dello stato ritorna al punto di equilibrio, allora si dice che l'equilibrio è asintoticamente stabile.

Nel caso di sistemi tempo lineari tempo invarianti del tipo:
\begin{cases}
\dot{x} = Ax +Bu \\
y = Cx + Du,
\end{cases}

è possibile dimostrare che la stabilità di un punto di equilibrio implica la stabilità di tutti i punti di equilibrio, quindi si può parlare di stabilità del sistema anche se, in generale, la proprietà di stabilità è correlata ad un punto di equilibrio. La caratteristica peculiare di questo sistema lineare è dovuta al fatto che l'evoluzione di questo tipo di sistemi è strettamente correlata agli autovalori della matrice dinamica $A$, che sono invarianti per rotazione, traslazione, condizione iniziale e tempo.

Ricordando quanto spiegato nell'esempio [Analisi modale](SS-02-Modal_Analysis.ipynb):

> La soluzione in forma chiusa dell'equazione differenziale, dal tempo iniziale $t_0$, con la condizione iniziale $x(t_0)$ è:
$$
x(t) = e^{A(t-t_0)}x(t_0).
$$ La matrice $e^{A(t-t_0)}x(t_0)$ è composta da combinazioni lineari di funzioni del tempo $t$, ciascuna del tipo: $$e^{\lambda t},$$ dove i $\lambda$ sono gli autovalori della matrice $A$; queste funzioni sono i modi del sistema.

così:
- un sistema dinamico lineare è stabile se e solo se tutti i suoi modi non sono divergenti,
- un sistema dinamico lineare è asintoticamente stabile se e solo se tutti i suoi modi sono convergenti,
- un sistema dinamico lineare è instabile se ha almeno un modo divergente.

e, considerando gli autovalori della matrice dinamica, queste condizioni valgono, rispettivamente, se:
- tutti gli autovalori di $A$ appartengono al semipiano complesso sinistro <u>chiuso</u> (cioè hanno parte reale negativa o nulla) e, nel caso in cui abbiano parte reale zero, la loro molteplicità algebrica è uguale alla molteplicità geometrica, o, equivalentemente, hanno blocchi scalari nella forma Jordan;
- tutti gli autovalori appartengono al semipiano complesso sinistro <u>aperto</u>, cioè hanno parte reale strettamente negativa;
- almeno un autovalore ha una parte reale positiva, o esistono autovalori con parte reale nulla e blocchi di Jordan non scalari.

Questo esempio interattivo presenta una matrice dinamica modificabile $A$ e mostra la risposta libera del sistema con gli autovalori corrispondenti.

### Come usare questo notebook?
- Prova a modificare gli autovalori e la condizione iniziale $x_0$ e osserva come cambia la risposta.

In [2]:
%matplotlib inline
#%matplotlib notebook 
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell

A = numpy.matrix([[0,1],[-2/5,-1/5]])
X0 = numpy.matrix('5; 3')

Aw = matrixWidget(2,2)
Aw.setM(A)
X0w = matrixWidget(2,1)
X0w.setM(X0)

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

In [5]:
# Main cell

def main_callback(A, X0, DW):
    sols = numpy.linalg.eig(A)
    sys = sss(A,[[1],[0]],[0,1],0)
    pole = control.pole(sys)
    if numpy.real(pole[0]) != 0:
        p1r = abs(numpy.real(pole[0]))
    else:
        p1r = 1
    if numpy.real(pole[1]) != 0:
        p2r = abs(numpy.real(pole[1]))
    else:
        p2r = 1
    if numpy.imag(pole[0]) != 0:
        p1i = abs(numpy.imag(pole[0]))
    else:
        p1i = 1
    if numpy.imag(pole[1]) != 0:
        p2i = abs(numpy.imag(pole[1]))
    else:
        p2i = 1
    
    print('Gli autovalori di A sono:',round(sols[0][0],4),'e',round(sols[0][1],4))
    
    #T = numpy.linspace(0, 60, 1000)
    T, yout, xout = control.initial_response(sys,X0=X0,return_x=True)
    
    fig = plt.figure("Autovalori di A", figsize=(16,16))
    ax = fig.add_subplot(311,title='Poli (Re vs Im)')
    #plt.axis(True)
    # Move left y-axis and bottim x-axis to centre, passing through (0,0)
    # Eliminate upper and right axes
    ax.spines['left'].set_position(('data',0.0))
    ax.spines['bottom'].set_position(('data',0.0))
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.set_xlim(-max([p1r+p1r/3,p2r+p2r/3]),
                max([p1r+p1r/3,p2r+p2r/3]))
    ax.set_ylim(-max([p1i+p1i/3,p2i+p2i/3]),
                max([p1i+p1i/3,p2i+p2i/3]))
    
    plt.plot([numpy.real(pole[0]),numpy.real(pole[1])],[numpy.imag(pole[0]),numpy.imag(pole[1])],'o')
    plt.grid()

    ax1 = fig.add_subplot(312,title='Risposta libera')
    plt.plot(T,xout[0])
    plt.grid()
    ax1.set_xlabel('t [s]')
    ax1.set_ylabel('$x_1$')
    ax1.axvline(x=0,color='black',linewidth='0.8')
    ax1.axhline(y=0,color='black',linewidth='0.8')
    ax2 = fig.add_subplot(313)
    plt.plot(T,xout[1])
    plt.grid()
    ax2.set_xlabel('t [s]')
    ax2.set_ylabel('$x_2$')
    ax2.axvline(x=0,color='black',linewidth='0.8')
    ax2.axhline(y=0,color='black',linewidth='0.8')
    
    #plt.show()
    

    
alltogether = widgets.HBox([widgets.VBox([widgets.Label('$A$:',border=3),
                                          Aw]),
                                          widgets.Label('   ',border=3),
                            widgets.VBox([widgets.Label('$X_0$:',border=3),
                                          X0w]),
                                          START])
out = widgets.interactive_output(main_callback, {'A':Aw, 'X0':X0w, 'DW':DW})
out.layout.height = '1000px'
display(out, alltogether)

Output(layout=Layout(height='1000px'))